# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [1]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [2]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [3]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [4]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [5]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [6]:
def query_planilha_embriologia(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia 
    WHERE PIN = {prontuario}
    ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "embryoscope_micro_prontuario" = {prontuario} OR "planilha_PIN" = {prontuario}
    ORDER BY "embryoscope_descong_em_DataTransferencia" DESC, "planilha_DATA_DA_FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


## Main Query Function


In [7]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia = query_planilha_embriologia(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia: {len(df_planilha_embriologia)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia': df_planilha_embriologia,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [8]:
# Example usage
prontuario = 823589

# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 823589
Query executed at: 2025-11-21 13:55:13.868661

=== VIEW_MICROMANIPULACAO ===
Found 1 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 12 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 2 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 1 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 1 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 6 records in view_embrioes_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 12 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 10 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===
Found 12 records in gold.embryoscope_clinisys_combined

=== SILVER.PLANILHA_EMBRIOLOGIA ===
Found 1 records in silver.planilha_embriologia

=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Found 12 records in gold.planilha_embryoscope_combined

QUERY SUMMARY FOR PRONTUARIO 823589
- view_micromanipulacao: 1 records
- view_micromanipulacao_oocitos: 12 records
- view_tratamentos: 2 records
- view_congelamentos_embrioes: 1 records
- view_descongelamentos_embrioes: 1 records
- view_embrioes_congelados: 6 records
- gold.clinisys_embrioes: 12 records
- gold.embryoscope_embrioes: 10 records
- gold.embryoscope_clinisys_combined: 12 records
- silver.planilha_embriologia: 1 records
- gold.planilha_embryoscope_combined: 12 records

Database connections closed.


## Display Results

### 1. Micromanipulacao


In [9]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 1


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,recepcao_ovulos,Denud_DL,checagem_fertilizacao,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,ICSIDescongelados,pgd_oocito,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,unidade_id,responsavel_denud_id,profissional_fertilizacao_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,materiais_insumos,local_coleta,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,observacoes_resumo_ciclos,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,conc_ps1,EstadoSptz,check_gradiente_ps,check_swimup,volume_ps2,concentracao_progressivos_pos,conc_ps2,comentarios_ps,responsavel_ps,timelapse,transferidos,Transfer_D5,medico_responsavel_id,azoospermia,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,19404,458/24,823589,38,39,2024-02-18,12:45,13:00,10:10,31,4318,Não,12:30,08:00,08:00,08:00,08:00,08:00,11,12,Não,1,1,2024-02-18,ICSI,10 espermatozoides móveis progressivos\r\n,10 espermatozoides normais\r\n\r\n,None,0000000001,3652,5997,3505,5997,4005,4005,None,Laboratório,Normal,"0,85",02,03,1,None,None,None,None,None,None,None,ES 1,03,None,None,Ejaculado,10:46,11:30,"2,4",Não,"2,6","42,0",Fresco,None,None,"0,2","3,0","3,0",None,3529,Sim,0,Sim,31,Nenhum,None,None,None,esposa,10,10,3,0,10,0,0,0,0,0,3,0,0,0,0,0,0,IB233/24,None,1cb78ba62ec30ea9ef7b1179e2ad50e2,2025-07-21 21:39:47


### 2. Micromanipulacao Oocitos


In [10]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 12


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,Embriologista,PI,TCD,PGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,OocitoDoado,OrigemOocito,InseminacaoOocito,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,NClivou_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,Compactando_D6,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,184366,19404,Não,MII,None,3505,None,Descartado,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,7,3,Simétricos,Sim,Blastocisto Grau 1,None,None,None,Blastocisto Grau 2,88a64fb9adc0d3ee1cfe6d5a1ec8ab96,2025-07-21 21:40:02,1,1
1,184367,19404,Não,MII,None,3505,3548,Criopreservado,Sim,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,7,2,Simétricos,Sim,Blastocisto Grau 5,B,B,None,None,307747a99b3ce48c0f88103900facd38,2025-07-21 21:40:02,1,2
2,184368,19404,Não,MII,None,3505,3548,Criopreservado,Sim,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,8,2,Simétricos,Sim,Blastocisto Grau 5,A,A,None,None,f00ec94e0184f821c1ae0ab0a81df525,2025-07-21 21:40:02,1,3
3,184369,19404,Não,MII,None,3505,3548,Criopreservado,Sim,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,Mórula A,None,None,Sim,Blastocisto Grau 4,A,B,None,None,2f8865a770f7d4158423d38debd70885,2025-07-21 21:40:02,1,4
4,184370,19404,Não,MII,None,3505,3548,Criopreservado,Sim,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,10,1,Simétricos,Sim,Blastocisto Grau 5,A,A,None,None,38743eadb99c8b299e17fd13307dd4a7,2025-07-21 21:40:02,1,5
5,184371,19404,Não,MII,None,3505,None,Descartado,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,4,3,Simétricos,Sim,Fragmentação 3,None,None,Não,None,a2b07d5946fbbabb1411bb49c8a2e723,2025-07-21 21:40:02,1,6
6,184372,19404,Não,MII,None,3505,None,Descartado,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,5,3,Simétricos,Não,None,None,None,None,None,bb778c154be5838477ecb1e106eeacc4,2025-07-21 21:40:02,1,7
7,184373,19404,Não,MII,None,3505,3548,Criopreservado,Sim,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,Mórula B,None,None,Sim,Blastocisto Grau 5,B,B,None,None,a614245834417811d0a0005a65f699fc,2025-07-21 21:40:02,1,8
8,184374,19404,Não,MII,None,3505,None,Descartado,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,Mfrag,None,None,Não,None,None,None,None,None,0e9173d0ebea59180e1f44e58a2b6cbb,2025-07-21 21:40:02,1,9
9,184375,19404,Não,MII,None,3505,5997,Criopreservado,Sim,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,6,2,Simétricos,Sim,Fragmentação 2,None,None,Sim,Blastocisto Grau 4,d3a859658624fca8a5e58c7780d0fddd,2025-07-21 21:40:02,1,10


### 3. Tratamentos


In [11]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 2


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,tipo_procedimento,tipo_inducao,protocolo,tipo_preparo_endometrial,origem_ovulo,status_ovulo,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,utero_substituicao,doacao_ovulos,doacao_embrioes,pgt,era_auxiliar,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,foliculos_antrais,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,data_inicio_agonista,utilizado_hcg,data_hcg,horario_planejado_hcg,gonapeptyl,dose_gonapeptyl,data_inicio_progesterona,progesterona_vaginal,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,motivo_nao_transferir,resultado_tratamento,analise_genetica,altura_bebe1,via_parto_bebe1,altura_bebe2,via_parto_bebe2,altura_bebe3,via_parto_bebe3,altura_bebe4,via_parto_bebe4,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,26793,823589,1,38,40,esposa,2,2024-05-10,Ciclo de Congelados,None,None,Natural,None,None,None,None,None,Homólogo,Não,None,None,PGT-A,Não,Não,Não,None,None,None,None,51kg,"1,60",None,2024-04-22,None,None,None,Fator masculino,Outros,None,2024-04-22,19,esposa,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,5,10/05/2024,None,None,None,Gestação Clínica,Não,None,None,None,None,None,None,None,None,None,3673,31,be2bbe8faa5f21d642725433e5ba3ac5,2025-07-21 21:38:38
1,25092,823589,1,38,39,esposa,1,2024-02-18,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Fresco,Homólogo,Sêmen,Fresco,None,Não,Não,Não,PGT-A,None,None,None,None,None,Sim,None,51kg,"1,60",None,2024-02-03,None,None,None,Outros,Fator masculino,None,2024-02-05,None,esposa,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,No transfer,None,None,None,Criopreservação total,No transfer,Não,None,None,None,None,None,None,None,None,None,3692,31,813de96ef73dc37eaec6e22a138d19b9,2025-07-21 21:38:38


### 4. Congelamentos Embrioes


In [12]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,D5,D6,Observacoes,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,24675,E451/24,1,823589,esposa,2024-02-23,1900-01-01 12:00:00,458/24,Não,6,6,E20747,MU,2,None,Vitrificação,1.0,1.0,None,Izadora Reis,Priscila Fabiane Dos Santos Beirigo Paternostro,None,76ed5aa97b623c19ed024b6a6b9bd68e,2025-07-21 21:38:47


### 5. Descongelamentos Embrioes


In [13]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 1


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,DataTransferencia,Incubadora,transferidos_transferencia,cateter_transferencia,lote_transferencia,validade_transferencia,sangue_interno_transferencia,sangue_externo_transferencia,retorno_transferencia,responsavel_transferencia,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,14763,980/24,1,823589,0,2024-02-23,2024-05-10,458/24,E20747,E451/24,MU,2,1,DESVITRIFICAÇÃO,2024-05-10,12.9,1,SYDNEY,None,None,Não,Não,Não,3719.0,None,None,None,c03d15bf60779ba46056f11d7c697ffe,2025-07-21 21:38:51


### 6. Embrioes Congelados


In [14]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    # display(embrioes_congelados_data)
    display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 6


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,embriao,qualidade,comentarios,comentarios_descongelamento,ah,dia5_2,dia5_3,dia5_4,transferidos,pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
2,69969,184370,24675,14763,823589,P5,E20747,#5,Blastocisto Grau 5 - A - A,None,"EUPLOIDE, Feminino",Sim,Blastocisto Grau 5,A,A,Transferido,Sim,Normal/Euplóide Feminino,Sim,Normal,9.7,D5,2.58,None,None,None,59d3eb2ab5662afda897880d4d7cc2ab,2025-07-21 21:38:57


### 7. Clinisys Embrioes


In [15]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(10))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 12


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_RC,oocito_Embriologista,oocito_TCD,oocito_PGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_Compactando_D6,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,id_1,cong_em_CodCongelamento,cong_em_Unidade,prontuario_1,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,Observacoes_1,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,id_2,descong_em_CodDescongelamento,Unidade_1,prontuario_2,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,Ciclo_1,Identificacao_1,CodCongelamento_1,Tambor_1,Cane_1,descong_em_PailletesDescongeladas,Tecnica_1,descong_em_DataTransferencia,Incubadora_1,descong_em_transferidos_transferencia,descong_em_cateter_transferencia,descong_em_lote_transferencia,descong_em_validade_transferencia,descong_em_sangue_interno_transferencia,descong_em_sangue_externo_transferencia,descong_em_retorno_transferencia,descong_em_responsavel_transferencia,Observacoes_2,BiologoFIV_1,BiologoFIV2_1,id_3,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,prontuario_3,emb_cong_pailletes,emb_cong_pailletes_id,emb_cong_embriao,emb_cong_qualidade,emb_cong_c

In [16]:
clinisys_embrioes_data[clinisys_embrioes_data['oocito_id_micromanipulacao'].isin([20386])]

,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_RC,oocito_Embriologista,oocito_TCD,oocito_PGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_Compactando_D6,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,id_1,cong_em_CodCongelamento,cong_em_Unidade,prontuario_1,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,Observacoes_1,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,id_2,descong_em_CodDescongelamento,Unidade_1,prontuario_2,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,Ciclo_1,Identificacao_1,CodCongelamento_1,Tambor_1,Cane_1,descong_em_PailletesDescongeladas,Tecnica_1,descong_em_DataTransferencia,Incubadora_1,descong_em_transferidos_transferencia,descong_em_cateter_transferencia,descong_em_lote_transferencia,descong_em_validade_transferencia,descong_em_sangue_interno_transferencia,descong_em_sangue_externo_transferencia,descong_em_retorno_transferencia,descong_em_responsavel_transferencia,Observacoes_2,BiologoFIV_1,BiologoFIV2_1,id_3,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,prontuario_3,emb_cong_pailletes,emb_cong_pailletes_id,emb_cong_embriao,emb_cong_qualidade,emb_cong_c

### 8. Embryoscope Embrioes


In [17]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 10


,prontuario,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_Comment,embryo_Time_Comment,embryo_Timestamp_Comment,embryo_Value_Comment,embryo_Name_DynamicScore,embryo_Time_DynamicScore,embryo_Timestamp_DynamicScore,embryo_Value_DynamicScore,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_Ellipse,embryo_Time_Ellipse,embryo_Timestamp_Ellipse,embryo_Value_Ellipse,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MN4Type,embryo_Time_MN4Type,embryo_Timestamp_MN4Type,embryo_Value_MN4Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MorphologicalGradeD5,embryo_Time_MorphologicalGradeD5,embryo_Timestamp_MorphologicalGradeD5,embryo_Value_MorphologicalGradeD5,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_Nuclei4,embryo_Time_Nuclei4,embryo_Timestamp_Nuclei4,embryo_Value_Nuclei4,embryo_Name_Nuclei8,embryo_Time_Nuclei8,embryo_Timestamp_Nuclei8,embryo_Value_Nuclei8,embryo_Name_Nuclei,embryo_Time_Nuclei,embryo_Timestamp_Nuclei,embryo_Value_Nuclei,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_Text,embryo_Time_Text,embryo_Timestamp_Text,embryo_Value_Text,embryo_Name_ZScore,embryo_Time_ZScore,embryo_Timestamp_ZScore,embryo_Value_ZScore,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_Name_Arrow,embryo_Time_Arrow,embryo_Timesta

### 9. Embryoscope Clinisys Combined


In [18]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(10))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 12


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_RC,oocito_Embriologista,oocito_TCD,oocito_PGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_Compactando_D6,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,id_1,cong_em_CodCongelamento,cong_em_Unidade,prontuario_1,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,Observacoes_1,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,id_2,descong_em_CodDescongelamento,Unidade_1,prontuario_2,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,Ciclo_1,Identificacao_1,CodCongelamento_1,Tambor_1,Cane_1,descong_em_PailletesDescongeladas,Tecnica_1,descong_em_DataTransferencia,Incubadora_1,descong_em_transferidos_transferencia,descong_em_cateter_transferencia,descong_em_lote_transferencia,descong_em_validade_transferencia,descong_em_sangue_interno_transferencia,descong_em_sangue_externo_transferencia,descong_em_retorno_transferencia,descong_em_responsavel_transferencia,Observacoes_2,BiologoFIV_1,BiologoFIV2_1,id_3,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,prontuario_3,emb_cong_pailletes,emb_cong_pailletes_id,emb_cong_embriao,emb_cong_qualidade,emb_cong_c

### 11. Planilha Embriologia


In [19]:
print("=== PLANILHA EMBRIOLOGIA DATA ===")
planilha_data = results['planilha_embriologia']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA ===
Records: 1


,PIN,Aux,RESP PREENCHIMENTO,UNIDADE,TIPO DE TRATAMENTO,TIPO DE FET,TIPO BIÓPSIA,TIPO DA DOAÇÃO,DATA INICIO PREPARO ENDOMETRIAL,DATA DA FET,MÊS,NOME DA PACIENTE,STATUS DO RELACIONAMENTO,FONTE PAGADORA,MÉDICO,GRUPO DE IDADE,IDADE MULHER,IDADE DO CONG DE EMBRIÃO,Data do congelamento,ALTURA,PESO,DATA DE NASC.,IDADE ESPERMATOZOIDE,FATOR 1,FATOR 2,PREPARO PARA TRANSFERENCIA,VIA FASE FOLICULAR,VIA LUTHEAL FASE,END. ESPESSURA,ESTAGIO BIOPSIA,Nº BIOPSIADOS,QTD ANALISADOS,QTD FALHA LEITURA,QTD NORMAIS,RESP CONGELAMENTO,QTD EMBRIO PREVIAMENTE CONGELADOS,DIA CRYO,ESTAGIO DESENVOLVIMENTO,DATA CRIO,MÉTODO DE CRIOPRESERVAÇÃO,RESP. DESCONGELAMENTO,Nº DESCONG,Nº SOBREVIVERAM,ESTAGIO DESENVOLVIMENTO3,Nº EMBR RECONGELADOS,Nº TOTAL EMBR. PERMANECEM CONG,RESP. TRANSFERENCIA,HOUVE TRANSFERENCIA?,Embryoglue,Qualidade Embrião transferido,"NÚMERO DA TRANSFER (1ª, 2ª, 3ª...)",DIA ET,ESTAGIO ET,Nº ET,MÉDICO ET,CATÉTER,ASSISTED HATCHING,INTERCORRÊNCIAS,RESULT,TIPO DO RESULTADO,OBS,GRAVIDEZ BIOQUIMICA,GRAVIDEZ CLINICA,DATA 1º US,SG 1º US,Qtd BCF,DATA 2º US,SG 2º US,Qtd BCF2,BCF,EVOLUÇÃO,DATA ABORTO,DATA DIAGNOSTICO GRAV. ECTOPICA,TIPO DE TRATAMENTO GRAV. ECTOP.,DATA INTERRUPÇÃO GRAV. ECTOP.,ESTUDO CITOGENÉTICO,RESULTADO DO ESTUDO GENÉTICO,ID GESTACIONAL,DATA PARTO,TIPO DE PARTO,Nº NASCIDOS,VIABILIDADE BEBÊ 1,PESO 1,ANOMALIA CONGÊNITA,ESTUDO CITOGEN,VIABILIDADE BEBÊ 2,PESO 2,ANOMALIA CONGÊN,ESTUDO CITOGE,VIABILIDADE BEBÊ 3,PESO 3,ANOMALIA CONG,ESTUDO CITOG,VIABILIDADE BEBÊ 4,PESO 4,ANOMALIA CONG.,ESTUDO CITOG.,OHSS,HEMORRAGIA,INFECÇÃO,Coluna1,ALTURA (cm),Auxiliar,Coluna12,IDADE DA MULHER,IDADE DO\n ESPERMATOZOIDE,Qtd BCF_1,line_number,extraction_timestamp,file_name
0,823589,None,LAURA BELOMO,IBIRAPUERA,FET FROM PGT,CYCLE CONTINUATION,PGT-A,None,2024-04-22,2024-05-10,MAIO,JULIANA UETSUKI MARTARELLI,None,None,Thais Domingues,3,38,38,None,160.0,51.0,1985-09-03,<NA>,MALE FACTOR,None,HORMONAL REPLACEMENT CYCLE,ORAL,Combined,7.1,None,<NA>,<NA>,<NA>,<NA>,Izadora Reis,3,5000000.0,BLASTOCYST,2024-02-23,VITRIFICATION,Priscila Paternostro,1,1,BLASTOCYST,<NA>,<NA>,Priscila Paternostro,1,None,None,1,5,BLASTOCYST,1,Thais Domingues,SYDNEY,0,None,EMBRYO TRANSFER,POSITIVO,ET COM ANESTESIA,1,1,NaT,1,1,NaT,<NA>,None,POSITIVO,<NA>,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,<NA>,None,<NA>,291,20251119_174507,CASOS 2024 IBIRA.xlsx


### 12. Planilha Embryoscope Combined


In [20]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data)
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===
Records: 12


,embryoscope_oocito_id,embryoscope_oocito_id_micromanipulacao,embryoscope_oocito_diaseguinte,embryoscope_oocito_Maturidade,embryoscope_oocito_RC,embryoscope_oocito_Embriologista,embryoscope_oocito_TCD,embryoscope_oocito_PGD,embryoscope_oocito_Fertilizacao,embryoscope_oocito_CorpusculoPolar,embryoscope_oocito_ComentariosDepois,embryoscope_oocito_OocitoDoado,embryoscope_oocito_OrigemOocito,embryoscope_oocito_InseminacaoOocito,embryoscope_oocito_NClivou_D3,embryoscope_oocito_NCelulas_D3,embryoscope_oocito_Frag_D3,embryoscope_oocito_Blastomero_D3,embryoscope_oocito_NClivou_D5,embryoscope_oocito_Compactando_D5,embryoscope_oocito_MassaInterna_D5,embryoscope_oocito_Trofoblasto_D5,embryoscope_oocito_NClivou_D6,embryoscope_oocito_Compactando_D6,embryoscope_oocito_flag_embryoscope,embryoscope_oocito_embryo_number,embryoscope_micro_codigo_ficha,embryoscope_micro_numero_caso,embryoscope_micro_prontuario,embryoscope_micro_IdadeEsposa_DG,embryoscope_micro_IdadeMarido_DG,embryoscope_micro_Data_DL,embryoscope_micro_horario_inicial_fert,embryoscope_micro_horario_final_fert,embryoscope_micro_Aspiracao_DL,embryoscope_micro_medico_id,embryoscope_micro_responsavel_puncao_id,embryoscope_micro_recepcao_ovulos,embryoscope_micro_Denud_DL,embryoscope_micro_checagem_fertilizacao,embryoscope_micro_checagem_d3,embryoscope_micro_checagem_d5,embryoscope_micro_checagem_d6,embryoscope_micro_checagem_d7,embryoscope_micro_foliculos,embryoscope_micro_oocitos,embryoscope_micro_ICSIDescongelados,embryoscope_micro_pgd_oocito,embryoscope_micro_pgd_d5,embryoscope_micro_data_procedimento,embryoscope_micro_tecnica_injetados,embryoscope_micro_motilidade,embryoscope_micro_morfologia_injetados,embryoscope_micro_gradiente_descontinuo,embryoscope_micro_unidade_id,embryoscope_micro_responsavel_denud_id,embryoscope_micro_profissional_fertilizacao_id,embryoscope_micro_profissional_d3_id,embryoscope_micro_profissional_d5_id,embryoscope_micro_profissional_d6_id,embryoscope_micro_profissional_d7_id,embryoscope_micro_materiais_insumos,embryoscope_micro_local_coleta,embryoscope_micro_viscosidade,embryoscope_micro_volume_amostra_utilizada,embryoscope_micro_morfologia_pre,embryoscope_micro_morfologia_pos,embryoscope_micro_zymot,embryoscope_micro_observacoes_resumo_ciclos,embryoscope_micro_agulha_icsi,embryoscope_micro_lote_agulha_icsi,embryoscope_micro_validade_agulha_icsi,embryoscope_micro_agulha_aspir,embryoscope_micro_lote_agulha_aspir,embryoscope_micro_validade_agulha_aspir,embryoscope_micro_incubadora,embryoscope_micro_abstinencia_dias,embryoscope_micro_meios,embryoscope_micro_responsavel_oocitos_id,embryoscope_micro_fonte,embryoscope_micro_inicio_ps,embryoscope_micro_termino_ps,embryoscope_micro_volume_ps1,embryoscope_micro_CongelamentoOvulos,embryoscope_micro_concentracao_progressivos_pre,embryoscope_micro_conc_ps1,embryoscope_micro_EstadoSptz,embryoscope_micro_check_gradiente_ps,embryoscope_micro_check_swimup,embryoscope_micro_volume_ps2,embryoscope_micro_concentracao_progressivos_pos,embryoscope_micro_conc_ps2,embryoscope_micro_comentarios_ps,embryoscope_micro_responsavel_ps,embryoscope_micro_timelapse,embryoscope_micro_transferidos,embryoscope_micro_Transfer_D5,embryoscope_micro_medico_responsavel_id,embryoscope_micro_azoospermia,embryoscope_micro_Observacoes,embryoscope_micro_responsavel_labfiv,embryoscope_micro_responsavel_labfiv_data,embryoscope_micro_tipo_paciente,embryoscope_micro_alteracoes_oocitarias_normal,embryoscope_micro_alteracoes_oocitarias_granulacao,embryoscope_micro_alteracoes_oocitarias_pelucida,embryoscope_micro_alteracoes_oocitarias_oval,embryoscope_micro_alteracoes_oocitarias_inclusoes,embryoscope_micro_alteracoes_oocitarias_pervitelineo_aumentado,embryoscope_micro_alteracoes_oocitarias_irregular,embryoscope_micro_alteracoes_oocitarias_necrotica,embryoscope_micro_alteracoes_oocitarias_pervitelineo_granular,embryoscope_micro_alteracoes_oocitarias_ser,embryoscope_micro_alteracoes_oocitarias_corpusculo_frag,embryoscope_micro_alteracoes_oocitarias_vacuolos,e